Parte 1

In [4]:
español = ['casa', 'perro', 'escuela', 'tarro', 'barro', 'carro']
ingles = ['house', 'dog', 'school', 'jar', 'mud', 'car']

def traductor(palabra):
    if palabra in español:
        indice = español.index(palabra)
        return ingles[indice]
    else:
        return "No sé"

palabra = input("Dime una palabra en español: ").lower()

traduccion = traductor(palabra)
print(f"La traduccion de '{palabra}' es: {traduccion}")

La traduccion de 'casa' es: house


Parte 2

In [3]:
import os

def cargar_diccionario():
    espanol, ingles = [], []
    if os.path.exists('diccionario.txt'):
        with open('diccionario.txt', 'r') as archivo:
            for linea in archivo:
                esp, ing = linea.strip().split(',')
                espanol.append(esp)
                ingles.append(ing)
    return espanol, ingles

def guardar_palabra(espanol, ingles):
    with open('diccionario.txt', 'a') as archivo:
        archivo.write(f'\n{espanol},{ingles}\n')

def traducir(palabra, espanol, ingles):
    if palabra in espanol:
        indice = espanol.index(palabra)
        return ingles[indice]
    else:
        return None 

espanol, ingles = cargar_diccionario()

palabra = input("Introduce una palabra en español: ").lower()

traduccion = traducir(palabra, espanol, ingles)

if traduccion:
    print(f"La traducción de '{palabra}' es: '{traduccion}'")
else:
    print(f"La palabra '{palabra}' no está en el diccionario.")
    nueva_traduccion = input(f"¿Cómo se dice '{palabra}' en inglés?: ").lower()
    
    espanol.append(palabra)
    ingles.append(nueva_traduccion)
    guardar_palabra(palabra, nueva_traduccion)
    
    print(f"Palabra '{palabra}' añadida al diccionario como '{nueva_traduccion}'.")

print("\nDiccionario actualizado:")
print(f"Español: {espanol}")
print(f"Inglés: {ingles}")

La traducción de 'arbol' es: 'tree'

Diccionario actualizado:
Español: ['casa', 'perro', 'escuela', 'tarro', 'barro', 'carro', 'arbol']
Inglés: ['house', 'dog', 'house', 'jar', 'mud', 'car', 'tree']


Parte 3

In [16]:
import os

def distancia_levenshtein(palabra1, palabra2):
    len1, len2 = len(palabra1), len(palabra2)
    matriz = [[0 for _ in range(len2 + 1)] for _ in range(len1 + 1)]
    
    for i in range(len1 + 1):
        matriz[i][0] = i
    for j in range(len2 + 1):
        matriz[0][j] = j
    
    for i in range(1, len1 + 1):
        for j in range(1, len2 + 1):
            if palabra1[i - 1] == palabra2[j - 1]:
                costo = 0
            else:
                costo = 1
            matriz[i][j] = min(matriz[i - 1][j] + 1,      
                               matriz[i][j - 1] + 1,     
                               matriz[i - 1][j - 1] + costo) 
    return matriz[len1][len2]

def sugerir_similitud(palabra, espanol, max_distancia=2):
    similares = []
    for palabra_dic in espanol:
        distancia = distancia_levenshtein(palabra, palabra_dic)
        if distancia <= max_distancia:
            similares.append(palabra_dic)
    return similares

def cargar_diccionario():
    espanol, ingles = [], []
    if os.path.exists('diccionario.txt'):
        with open('diccionario.txt', 'r') as archivo:
            for linea in archivo:
                partes = linea.strip().split(',')
                if len(partes) == 2:
                    esp, ing = partes
                    espanol.append(esp)
                    ingles.append(ing)
    return espanol, ingles

def guardar_palabra(espanol, ingles):
    with open('diccionario.txt', 'a') as archivo:
        archivo.write(f'\n{espanol},{ingles}')

def cargar_errores():
    errores, similitudes, contadores = [], [], []
    if os.path.exists('errores.txt'):
        with open('errores.txt', 'r') as archivo:
            for linea in archivo:
                partes = linea.strip().split(',')
                if len(partes) == 3: 
                    error, similitud, contador = partes
                    errores.append(error)
                    similitudes.append(similitud)
                    contadores.append(int(contador))
    return errores, similitudes, contadores

def guardar_error(error, similitud, contador):
    with open('errores.txt', 'a') as archivo:
        archivo.write(f'\n{error},{similitud},{contador}')

def actualizar_errores(errores, similitudes, contadores):
    with open('errores.txt', 'w') as archivo:
        for i in range(len(errores)):
            archivo.write(f'{errores[i]},{similitudes[i]},{contadores[i]}\n')

def traducir(palabra, espanol, ingles):
    if palabra in espanol:
        indice = espanol.index(palabra)
        return ingles[indice]
    else:
        return None

espanol, ingles = cargar_diccionario()
errores, similitudes, contadores = cargar_errores()

palabra = input("Introduce una palabra en español: ").lower()

traduccion = traducir(palabra, espanol, ingles)

if traduccion:
    print(f"La traducción de '{palabra}' es: '{traduccion}'")
else:
    if palabra in errores:
        indices_error = [i for i, error in enumerate(errores) if error == palabra]
        
        max_index = indices_error[0]
        for i in indices_error:
            if contadores[i] > contadores[max_index]:
                max_index = i
        
        similitud_elegida = similitudes[max_index]
        contadores[max_index] += 1
        
        print(f"Detecté el error '{palabra}'. La palabra correcta es '{similitud_elegida}'")
        actualizar_errores(errores, similitudes, contadores)
    else:
        similares = sugerir_similitud(palabra, espanol)
        if similares:
            print(f"Tal vez quisiste decir: {', '.join(similares)}?")
            eleccion = input("Escribe la palabra correcta: ").lower()
            
            if eleccion in similares:
                espanol.append(eleccion)
                ingles.append(traducir(eleccion, espanol, ingles))
                
                guardar_error(palabra, eleccion, 1)
                print(f"Aprendí que '{palabra}' es similar a '{eleccion}'.")
            else:
                print("No se reconoció la elección.")
        else:
            print(f"No encontré palabras similares a '{palabra}'.")

print("\nDiccionario actualizado:")
print(f"Español: {espanol}")
print(f"Inglés: {ingles}")


La traducción de 'casa' es: 'house'

Diccionario actualizado:
Español: ['casa', 'perro', 'escuela', 'tarro', 'barro', 'carro']
Inglés: ['house', 'dog', 'house', 'jar', 'mud', 'car']


Version 2

In [13]:
import os
import difflib

# Función para cargar las palabras desde un archivo
def cargar_diccionario():
    espanol, ingles = [], []
    if os.path.exists('diccionario.txt'):
        with open('diccionario.txt', 'r') as archivo:
            for linea in archivo:
                partes = linea.strip().split(',')
                if len(partes) == 2:
                    esp, ing = partes
                    espanol.append(esp)
                    ingles.append(ing)
    return espanol, ingles

# Función para guardar nuevas palabras en el archivo
def guardar_palabra(espanol, ingles):
    with open('diccionario.txt', 'a') as archivo:
        archivo.write(f'\n{espanol},{ingles}')

# Función para cargar los errores de similitud
def cargar_errores():
    errores, similitudes, contadores = [], [], []
    if os.path.exists('errores.txt'):
        with open('errores.txt', 'r') as archivo:
            for linea in archivo:
                partes = linea.strip().split(',')
                if len(partes) == 3:  # error, similitud, contador
                    error, similitud, contador = partes
                    errores.append(error)
                    similitudes.append(similitud)
                    contadores.append(int(contador))
    return errores, similitudes, contadores

# Función para guardar un nuevo error y similitud en el archivo
def guardar_error(error, similitud, contador):
    with open('errores.txt', 'a') as archivo:
        archivo.write(f'\n{error},{similitud},{contador}')

# Función para actualizar el archivo de errores cuando el contador aumenta
def actualizar_errores(errores, similitudes, contadores):
    with open('errores.txt', 'w') as archivo:
        for i in range(len(errores)):
            archivo.write(f'{errores[i]},{similitudes[i]},{contadores[i]}\n')

# Función para traducir de español a inglés
def traducir(palabra, espanol, ingles):
    if palabra in espanol:
        indice = espanol.index(palabra)
        return ingles[indice]
    else:
        return None

# Función para buscar palabras similares
def sugerir_similitud(palabra, espanol):
    return difflib.get_close_matches(palabra, espanol, n=2, cutoff=0.6)

# Cargar diccionario y errores previos
espanol, ingles = cargar_diccionario()
errores, similitudes, contadores = cargar_errores()

# Solicitar una palabra en español
palabra = input("Introduce una palabra en español: ").lower()

# Verificar si la palabra existe en el diccionario
traduccion = traducir(palabra, espanol, ingles)

if traduccion:
    print(f"La traducción de '{palabra}' es: '{traduccion}'")
else:
    # Buscar si el error ya ha ocurrido antes
    if palabra in errores:
        indices_error = [i for i, error in enumerate(errores) if error == palabra]
        
        # Buscar la palabra similar con el mayor contador
        max_index = indices_error[0]
        for i in indices_error:
            if contadores[i] > contadores[max_index]:
                max_index = i
        
        similitud_elegida = similitudes[max_index]
        contadores[max_index] += 1
        
        print(f"Detecté el error '{palabra}'. La palabra correcta es '{similitud_elegida}'")
        actualizar_errores(errores, similitudes, contadores)
    else:
        # Si es un error nuevo, sugerimos palabras similares
        similares = sugerir_similitud(palabra, espanol)
        if similares:
            print(f"Tal vez quisiste decir: {', '.join(similares)}?")
            eleccion = input("Escribe la palabra correcta: ").lower()
            
            if eleccion in similares:
                # Agregar la palabra similar elegida y guardarla
                espanol.append(eleccion)
                ingles.append(traducir(eleccion, espanol, ingles))
                
                # Guardar el error con la nueva similitud y contador inicial de 1
                guardar_error(palabra, eleccion, 1)
                print(f"Aprendí que '{palabra}' es similar a '{eleccion}'.")
            else:
                print("No se reconoció la elección.")
        else:
            print(f"No encontré palabras similares a '{palabra}'.")

# Mostrar los diccionarios actualizados
print("\nDiccionario actualizado:")
print(f"Español: {espanol}")
print(f"Inglés: {ingles}")


Detecté el error 'karro'. La palabra correcta es 'carro'

Diccionario actualizado:
Español: ['casa', 'perro', 'escuela', 'tarro', 'barro', 'carro']
Inglés: ['house', 'dog', 'house', 'jar', 'mud', 'car']
